# **Scrapping Algorithm**

In [13]:
from bs4 import BeautifulSoup  # to parse html
from datetime import datetime
import os
import pandas as pd
from pip._vendor import requests
from time import (sleep,)  # function that allows you to pause the program. helpful to spae out requests

In [14]:
# Define our url: https://www.failory.com/startups/germany

html_text = requests.get('https://www.failory.com/startups/germany', timeout=15)

In [15]:
# Getting the info from the beautifulSoup
soup = BeautifulSoup(html_text.content, 'html.parser')
startups = soup.find_all('h3')
lists = soup.find_all('ul')
startups = [str(i) for i in startups]
lists = [str(i) for i in lists]
lists = [i.replace('</li><li>', '</li>  <li>') for i in lists]

In [16]:
startups = [i.strip("<h3>/)1234567890 ") for i in startups]

In [17]:
#Cleaning all the html stuff
import re
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):  #Function given I'm working on disentangling what ".sub" does... to be continued
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext
lists = [cleanhtml(lists[i]) for i in range(len(lists))]

In [18]:
lists = [i.split("  ") for i in lists]
lists[0]

['City: Munich',
 'Started in: 2011',
 'Founders: Alexander Rinke, Bastian Nominacher, Martin Klenk',
 'Industries: Analytics, Business Intelligence, SaaS, Software',
 'Number of employees: 1001-5000',
 'Funding amount: $1,367,500,000',
 'Number of funding rounds: 4',
 'Number of investors: 9']

In [19]:
for i, k in enumerate(startups):
    lists[i].insert(0, k)

In [23]:
lists_dict = []
for i in lists:
    job_dict = {}
    job_dict['Company'] = i[0]
    for k in i:
        if 'City' in k:
            k_rep = k.replace("City: ", "")
            job_dict['City'] = k_rep
        elif 'Started in' in k:
            k_rep = k.replace("Started in: ", "")
            job_dict['Started_in'] = k_rep
        elif 'Founders:' in k:
            k_rep = k.replace("Founders: ", "")
            job_dict['Founders'] = k_rep
        elif 'Industries:' in k:
            k_rep = k.replace("Industries: ", "")
            job_dict['Industries'] = k_rep
        elif 'Number of employees:' in k:
            k_rep = k.replace("Number of employees: ", "")
            job_dict['Number_of_employees'] = k_rep
        elif 'Funding amount' in k:
            k_rep = k.replace("Funding amount: ", "")
            job_dict['Funding_amount'] = k_rep
        elif 'Number of funding rounds: ' in k:
            k_rep = k.replace("Number of funding rounds: ", "")
            job_dict['Number_of_funding_rounds'] = k_rep
        elif 'Number of investors: ' in k:
            k_rep = k.replace("Number of investors: ", "")
            job_dict['Number_of_investors'] = k_rep
        else:
            continue
    lists_dict.append(job_dict)

In [24]:
df = pd.DataFrame.from_dict(lists_dict)
df.to_csv('../data/raw/sturtups_in_germany.csv')